## EDA

In [1]:
import os
import time
import datetime
import json
import gc
from numba import jit

import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold,KFold
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from bayes_opt import BayesianOptimization
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn import metrics

from itertools import product

from IPython.display import HTML

In [2]:
folder_path = 'data/'
train_identity = pd.read_csv(f'{folder_path}train_identity.csv')
train_transaction = pd.read_csv(f'{folder_path}train_transaction.csv')
test_identity = pd.read_csv(f'{folder_path}test_identity.csv')
test_transaction = pd.read_csv(f'{folder_path}test_transaction.csv')
sub = pd.read_csv(f'{folder_path}sample_submission.csv')
# let's combine the data and work with the whole dataset
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

In [4]:
train_transaction[train_transaction.isFraud == 1].head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
203,2987203,1,89760,445.000,W,18268,583.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,2987240,1,90193,37.098,C,13413,103.0,185.0,visa,137.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,2987243,1,90246,37.098,C,13413,103.0,185.0,visa,137.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,2987245,1,90295,37.098,C,13413,103.0,185.0,visa,137.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288,2987288,1,90986,155.521,C,16578,545.0,185.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from WOE import *
from woe2 import *

final_iv, IV = data_vars(train,train.isFraud)

In [5]:
SelectedFeatures =  IV[IV.IV > 0.02].VAR_NAME.tolist()

In [10]:
train2.shape, test2.shape

((590540, 434), (506691, 433))

In [9]:
CategoricalFeatures = list(filter(lambda x:x in list(train.select_dtypes(include=['O']).columns),SelectedFeatures))
ContinuousFeatures = list(filter(lambda x:x not in list(train.select_dtypes(include=['O']).columns),SelectedFeatures))
ConvertCategorical = ConvertCategoricalFeatures("isFraud",CategoricalFeatures)
ConvertCategorical.train(train)
ConvertContinuous = ConvertContinuousFeatures("isFraud",ContinuousFeatures,10)
ConvertContinuous.train(train)
train2 = ConvertCategorical.transform(train)
train2 = ConvertContinuous.transform(train2)
test2 = ConvertCategorical.transform(test)
test2 = ConvertContinuous.transform(test2)

In [ ]:
for col, values in train_transaction[train_transaction.isFraud == 1].iteritems():
    num_uniques = values.nunique()
    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))
    print (values.unique())
    print ('\n')

In [38]:
#cut tr and val
tr_idx, val_idx = train_test_split(train, test_size = 0.3, random_state = 87, stratify = train['isFraud'])

In [41]:
tr_idx = tr_idx.index
val_idx = val_idx.index

## Simple XGB

In [80]:
y_train = train.isFraud
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()

In [81]:
X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)
# Label Encoding
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))   

In [53]:
tr_idx

Int64Index([506347, 377863,  27133,  68424, 481102, 452875, 437524, 471476,
            228760,  86640,
            ...
            121676, 356189, 442469, 138974, 428712, 319420, 410637, 253692,
            137909,  58306],
           dtype='int64', length=413378)

In [58]:
X_train_tr = X_train[X_train.index.isin(tr_idx)]
X_train_val = X_train[X_train.index.isin(val_idx)]
y_train_tr = y_train[y_train.index.isin(tr_idx)]
y_train_val = y_train[y_train.index.isin(val_idx)]

In [ ]:
xgb_raw = xgb.XGBClassifier(
    #objective ='binary:logistic',
    n_estimators=5000,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    missing=-999,
    nthread=10,
    eval_metric=["auc"])

xgb_raw.fit(X_train_tr,
           y_train_tr,
           eval_set=[(X_train_tr, y_train_tr),
                     (X_train_val, y_train_val)],
           early_stopping_rounds = 50,
           verbose=20
          )

[0]	validation_0-auc:0.778701	validation_1-auc:0.776197
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 50 rounds.
[20]	validation_0-auc:0.840296	validation_1-auc:0.836135
[40]	validation_0-auc:0.866693	validation_1-auc:0.862033
[60]	validation_0-auc:0.889208	validation_1-auc:0.88311
[80]	validation_0-auc:0.901699	validation_1-auc:0.893806
[100]	validation_0-auc:0.910865	validation_1-auc:0.900809
[120]	validation_0-auc:0.919003	validation_1-auc:0.907459
[140]	validation_0-auc:0.924548	validation_1-auc:0.912186
[160]	validation_0-auc:0.929027	validation_1-auc:0.91584
[180]	validation_0-auc:0.932154	validation_1-auc:0.918376
[200]	validation_0-auc:0.934761	validation_1-auc:0.920581
[220]	validation_0-auc:0.936945	validation_1-auc:0.92222
[240]	validation_0-auc:0.939457	validation_1-auc:0.924458
[260]	validation_0-auc:0.941396	validation_1-auc:0.925842
[280]	validation_0-auc:0.943248	validati

In [65]:
xgb_raw.best_score

0.971037

In [59]:
clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    missing=-999,
    nthread=10
    #tree_method='gpu_hist'  # THE MAGICAL PARAMETER
)

In [11]:
%time clf.fit(X_train, 
              y_train, 
              eval_set=None, eval_metric=None
              early_stopping_rounds = 100)

CPU times: user 2h 26min 58s, sys: 6min 24s, total: 2h 33min 22s
Wall time: 19min 22s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.9, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=-999, n_estimators=500, n_jobs=1,
       nthread=10, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=0.9, verbosity=1)

In [69]:
test.shape

(10000, 434)

In [82]:
sample_submission = pd.read_csv(f'{folder_path}sample_submission.csv', index_col='TransactionID')
sample_submission['isFraud'] = xgb_raw.predict_proba(X_test)[:,1]
sample_submission.to_csv('xgb_raw_4000r.csv')

## XGB with WOE 

In [11]:
y_train = train2.isFraud
X_train = train2.drop('isFraud', axis=1)
X_test = test2.copy()

X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)
# Label Encoding
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))   

In [ ]:
clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    missing=-999,
    nthread=10
    #tree_method='gpu_hist'  # THE MAGICAL PARAMETER
)

## LGBM with Bayes Opt

In [13]:
#  Libraries
import numpy as np 
import pandas as pd 
# Data processing, metrics and modeling
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold,KFold
from bayes_opt import BayesianOptimization
from datetime import datetime
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, roc_curve, auc,precision_recall_curve
from sklearn import metrics
from sklearn import preprocessing
# Lgbm
import lightgbm as lgb
# Suppr warning
import warnings
warnings.filterwarnings("ignore")

import itertools
from scipy import interp

# Plots
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams

In [14]:
features = list(train2)
features.remove('isFraud')
target = 'isFraud'

In [15]:
train = train2.fillna(-999)
test = test2.fillna(-999)
for f in features:
    if train[f].dtype=='object' or test[f].dtype=='object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values) + list(test[f].values))
        train[f] = lbl.transform(list(train[f].values))
        test[f] = lbl.transform(list(test[f].values))


In [16]:
#cut tr and val
bayesian_tr_idx, bayesian_val_idx = train_test_split(train, test_size = 0.3, random_state = 87, stratify = train[target])
bayesian_tr_idx = bayesian_tr_idx.index
bayesian_val_idx = bayesian_val_idx.index

In [17]:
def LGB_bayesian(
    #learning_rate,
    num_leaves, 
    bagging_fraction,
    feature_fraction,
    min_child_samples, 
    min_child_weight,
    subsample, 
    min_data_in_leaf,
    max_depth,
    colsample_bytree,
    reg_alpha,
    reg_lambda
     ):
    
    # LightGBM expects next three parameters need to be integer. 
    num_leaves = int(num_leaves)
    min_data_in_leaf = int(min_data_in_leaf)
    max_depth = int(max_depth)

    assert type(num_leaves) == int
    assert type(min_data_in_leaf) == int
    assert type(max_depth) == int
    

    param = {
              'num_leaves': num_leaves, 
              'min_child_samples': min_child_samples, 
              'min_data_in_leaf': min_data_in_leaf,
              'min_child_weight': min_child_weight,
              'bagging_fraction' : bagging_fraction,
              'feature_fraction' : feature_fraction,
              #'learning_rate' : learning_rate,
              'subsample': subsample, 
              'max_depth': max_depth,
              'colsample_bytree': colsample_bytree,
              'reg_alpha': reg_alpha,
              'reg_lambda': reg_lambda,
              'objective': 'binary',
              'save_binary': True,
              'seed': 1337,
              'feature_fraction_seed': 1337,
              'bagging_seed': 1337,
              'drop_seed': 1337,
              'data_random_seed': 1337,
              'boosting_type': 'gbdt',
              'verbose': 1,
              'is_unbalance': False,
              'boost_from_average': True,
              'metric':'auc'}    
    
    oof = np.zeros(len(train))
    trn_data= lgb.Dataset(train.iloc[bayesian_tr_idx][features].values, label=train.iloc[bayesian_tr_idx][target].values)
    val_data= lgb.Dataset(train.iloc[bayesian_val_idx][features].values, label=train.iloc[bayesian_val_idx][target].values)

    clf = lgb.train(param, trn_data,  num_boost_round=50, valid_sets = [trn_data, val_data], verbose_eval=0, early_stopping_rounds = 50)
    
    oof[bayesian_val_idx]  = clf.predict(train.iloc[bayesian_val_idx][features].values, num_iteration=clf.best_iteration)  
    
    score = roc_auc_score(train.iloc[bayesian_val_idx][target].values, oof[bayesian_val_idx])

    return score

In [18]:
# Bounded region of parameter space
bounds_LGB = {
    'num_leaves': (31, 500), 
    'min_data_in_leaf': (20, 200),
    'bagging_fraction' : (0.1,0.9),
    'feature_fraction' : (0.1,0.9),
    #'learning_rate': (0.01, 0.3),
    'min_child_weight': (0.00001, 0.01),   
    'min_child_samples':(100, 500), 
    'subsample': (0.2, 0.8),
    'colsample_bytree': (0.4, 0.6), 
    'reg_alpha': (1, 2), 
    'reg_lambda': (1, 2),
    'max_depth':(-1,50),
}

In [19]:
LGB_BO = BayesianOptimization(LGB_bayesian, bounds_LGB, random_state=87)

In [20]:
print(LGB_BO.space.keys)

['bagging_fraction', 'colsample_bytree', 'feature_fraction', 'max_depth', 'min_child_samples', 'min_child_weight', 'min_data_in_leaf', 'num_leaves', 'reg_alpha', 'reg_lambda', 'subsample']


In [21]:
init_points = 10
n_iter = 15

In [22]:
print('-' * 130)

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=init_points, n_iter=n_iter, acq='ucb', xi=0.0, alpha=1e-6)

----------------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   | baggin... | colsam... | featur... | max_depth | min_ch... | min_ch... | min_da... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.881    |  0.625    |  0.591    |  0.8767   |  3.989    |  196.3    |  0.008688 |  118.5    |  368.3    |  1.926    |  1.617    |  0.6001   |
|  2        |  0.9497   |  0.3671   |  0.4366   |  0.39     |  19.49    |  404.8    |  0.002675 |  68.08    |  488.8    |  1.062    |  1.419    |  0.4107   |
|  3        |  0.9435   |  0.1716   |  0.4395   |  0.8527   | -0.4612   |  174.4    |  0.009855 |  81.25    |  184.1    |  1.654    |  1.022    |  0.3152   |
|  4        |  0.9426   |  0.862    |  0.5501   |  0.2816   |  31.53    |  112.

In [23]:
LGB_BO.max['target']

0.9512627555278169

In [24]:
LGB_BO.max['params']

{'bagging_fraction': 0.38556105567612287,
 'colsample_bytree': 0.42002158959021396,
 'feature_fraction': 0.6447037456245331,
 'max_depth': 46.35144228764539,
 'min_child_samples': 109.11536049498326,
 'min_child_weight': 0.0094625996071842,
 'min_data_in_leaf': 172.36974218753306,
 'num_leaves': 495.51839992023577,
 'reg_alpha': 1.201633564319697,
 'reg_lambda': 1.524883249967171,
 'subsample': 0.6489206303665163}

In [6]:
param_lgb = {
        'min_data_in_leaf': int(21.427642007695642), 
        'num_leaves': int(490.15525281823756), 
        #'learning_rate': LGB_BO.max['params']['learning_rate'],
        'min_child_weight': 0.0031778570175284566,
        'colsample_bytree' : 0.5579195650937059,
        'bagging_fraction': 0.2548043210587658, 
        'min_child_samples': 299.06472700672117,
        'subsample': 0.7539309525325935,
        'reg_lambda': 1.143573447284028,
        'reg_alpha': 1.6083331773049245,
        'max_depth': 46, 
        'objective': 'binary',
        'save_binary': True,
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'boosting_type': 'gbdt',
        'verbose': 1,
        'is_unbalance': False,
        'boost_from_average': True,
        'metric':'auc'
    }

In [25]:
param_lgb = {
        'min_data_in_leaf': int(LGB_BO.max['params']['min_data_in_leaf']), 
        'num_leaves': int(LGB_BO.max['params']['num_leaves']), 
        #'learning_rate': LGB_BO.max['params']['learning_rate'],
        'min_child_weight': LGB_BO.max['params']['min_child_weight'],
        'colsample_bytree' : LGB_BO.max['params']['colsample_bytree'],
        'bagging_fraction': LGB_BO.max['params']['bagging_fraction'], 
        'min_child_samples': LGB_BO.max['params']['min_child_samples'],
        'subsample': LGB_BO.max['params']['subsample'],
        'reg_lambda': LGB_BO.max['params']['reg_lambda'],
        'reg_alpha': LGB_BO.max['params']['reg_alpha'],
        'max_depth': int(LGB_BO.max['params']['max_depth']), 
        'objective': 'binary',
        'save_binary': True,
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'boosting_type': 'gbdt',
        'verbose': 1,
        'is_unbalance': False,
        'boost_from_average': True,
        'metric':'auc'
    }

In [26]:
test.shape, ta

(506691, 433)

In [ ]:
plt.rcParams["axes.grid"] = True

nfold = 5
skf = StratifiedKFold(n_splits=nfold, shuffle=True, random_state=42)

oof = np.zeros(len(train))
mean_fpr = np.linspace(0,1,100)
cms= []
tprs = []
aucs = []
y_real = []
y_proba = []
recalls = []
roc_aucs = []
f1_scores = []
accuracies = []
precisions = []
predictions = np.zeros(len(test))
feature_importance_df = pd.DataFrame()

i = 1
for train_idx, valid_idx in skf.split(train, train.isFraud.values):
    print("\nfold {}".format(i))
    trn_data = lgb.Dataset(train.iloc[train_idx][features].values,
                                   label=train.iloc[train_idx][target].values
                                   )
    val_data = lgb.Dataset(train.iloc[valid_idx][features].values,
                                   label=train.iloc[valid_idx][target].values
                                   )   
    
    clf = lgb.train(param_lgb, trn_data, num_boost_round = 500, valid_sets = [trn_data, val_data], verbose_eval = 100, early_stopping_rounds = 100)
    oof[valid_idx] = clf.predict(train.iloc[valid_idx][features].values) 
    
    predictions += clf.predict(test[features]) / nfold
    
    # Scores 
    roc_aucs.append(roc_auc_score(train.iloc[valid_idx][target].values, oof[valid_idx]))
    accuracies.append(accuracy_score(train.iloc[valid_idx][target].values, oof[valid_idx].round()))
    recalls.append(recall_score(train.iloc[valid_idx][target].values, oof[valid_idx].round()))
    precisions.append(precision_score(train.iloc[valid_idx][target].values ,oof[valid_idx].round()))
    f1_scores.append(f1_score(train.iloc[valid_idx][target].values, oof[valid_idx].round()))
    
    # Roc curve by folds
    f = plt.figure(1)
    fpr, tpr, t = roc_curve(train.iloc[valid_idx][target].values, oof[valid_idx])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.4f)' % (i,roc_auc))
    
    # Precion recall by folds
    g = plt.figure(2)
    precision, recall, _ = precision_recall_curve(train.iloc[valid_idx][target].values, oof[valid_idx])
    y_real.append(train.iloc[valid_idx][target].values)
    y_proba.append(oof[valid_idx])
    plt.plot(recall, precision, lw=2, alpha=0.3, label='P|R fold %d' % (i))  
    
    i= i+1
    
    # Confusion matrix by folds
    cms.append(confusion_matrix(train.iloc[valid_idx][target].values, oof[valid_idx].round()))
    
    # Features imp
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = nfold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

# Metrics
print(
        '\nCV roc score        : {0:.4f}, std: {1:.4f}.'.format(np.mean(roc_aucs), np.std(roc_aucs)),
        '\nCV accuracy score   : {0:.4f}, std: {1:.4f}.'.format(np.mean(accuracies), np.std(accuracies)),
        '\nCV recall score     : {0:.4f}, std: {1:.4f}.'.format(np.mean(recalls), np.std(recalls)),
        '\nCV precision score  : {0:.4f}, std: {1:.4f}.'.format(np.mean(precisions), np.std(precisions)),
        '\nCV f1 score         : {0:.4f}, std: {1:.4f}.'.format(np.mean(f1_scores), np.std(f1_scores))
)

#ROC 
f = plt.figure(1)
plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'grey')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.4f)' % (np.mean(roc_aucs)),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('LGB ROC curve by folds')
plt.legend(loc="lower right")

# PR plt
g = plt.figure(2)
plt.plot([0,1],[1,0],linestyle = '--',lw = 2,color = 'grey')
y_real = np.concatenate(y_real)
y_proba = np.concatenate(y_proba)
precision, recall, _ = precision_recall_curve(y_real, y_proba)
plt.plot(recall, precision, color='blue',
         label=r'Mean P|R')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('P|R curve by folds')
plt.legend(loc="lower left")


fold 1
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.9951	valid_1's auc: 0.963442
[200]	training's auc: 0.999623	valid_1's auc: 0.969212
[300]	training's auc: 0.999972	valid_1's auc: 0.970079
[400]	training's auc: 0.999998	valid_1's auc: 0.970453
Early stopping, best iteration is:
[350]	training's auc: 0.999991	valid_1's auc: 0.970489

fold 2
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.995123	valid_1's auc: 0.963486
[200]	training's auc: 0.999656	valid_1's auc: 0.968925
[300]	training's auc: 0.999977	valid_1's auc: 0.970235
[400]	training's auc: 0.999998	valid_1's auc: 0.970765
[500]	training's auc: 1	valid_1's auc: 0.970857
Did not meet early stopping. Best iteration is:
[500]	training's auc: 1	valid_1's auc: 0.970857

fold 3
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.995087	valid_1's auc: 0.96317
[200]	training's auc: 0.999679	valid_1's auc: 0.969003
[300]	

In [29]:
predictions

array([0.00077443, 0.0013164 , 0.00138516, ..., 0.00072717, 0.00035343,
       0.00068623])

In [30]:
len(predictions)

506691

In [101]:
sample_submission = pd.read_csv(f'{folder_path}sample_submission.csv', index_col='TransactionID')

In [108]:
test2.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,-0.554564,13926,NaN,-0.353155,discover,142.0,...,-0.520989,-0.043373,-0.046728,NaN,-0.524896,-0.524896,-0.524896,-0.524896,-0.524104,-0.324192
1,2987001,0,86401,29.0,-0.554564,2755,404.0,-0.353155,mastercard,102.0,...,-0.520989,-0.043373,-0.046728,NaN,-0.524896,-0.524896,-0.524896,-0.524896,-0.524104,-0.324192
2,2987002,0,86469,59.0,-0.554564,4663,490.0,-0.353155,visa,166.0,...,-0.520989,-0.043373,-0.046728,NaN,-0.524896,-0.524896,-0.524896,-0.524896,-0.524104,-0.324192
3,2987003,0,86499,50.0,-0.554564,18132,567.0,-0.353155,mastercard,117.0,...,-0.520989,-0.043373,-0.046728,NaN,-0.524896,-0.524896,-0.524896,-0.524896,-0.524104,-0.324192
4,2987004,0,86506,50.0,0.322297,4497,514.0,-0.353155,mastercard,102.0,...,0.740908,0.678983,0.440205,match_status:2,0.254637,0.900070,0.918904,0.557373,1.138187,-2.29605


In [31]:
sample_submission = pd.read_csv(f'{folder_path}sample_submission.csv', index_col='TransactionID')
sample_submission['isFraud'] = predictions
sample_submission.to_csv('xbg_bayes_opt_woe.csv')